
## Trying with my classifier

In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, Conv2DTranspose
from keras.models import Model
from keras import backend as K
from PIL import Image
from tqdm import tqdm
import numpy as np
import tensorflow as ts
import matplotlib.pyplot as plt
from keras.models import load_model
from keras import 
from keras.callbacks import CSVLogger
import os 

In [ ]:
# CONVOLUTIONAL AUTOENCODER

input_img = Input(shape=(64, 64, 3))

x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img) #16
x = AveragePooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x) #8
x = AveragePooling2D((2, 2), padding='same')(x)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
                
x = Conv2DTranspose(32, (3, 3), strides=2, activation='relu', padding='same')(encoded)
x = Conv2DTranspose(64, (3, 3), strides=2, activation='relu', padding='same')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)


In [ ]:
# Load previously saved images using get_training_data function (can be found at the end of notebook)

images = np.load('humans_64.npy')

In [ ]:
images = images.astype('float32') / 255.
images.shape

In [ ]:
x_train = images[0:5000]
x_test = images[5000:6500]

x_train = np.reshape(x_train, (len(x_train), 64, 64, 3))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 64, 64, 3))  # adapt this if using `channels_first` image data format


In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
#  TO SAVE MEMORY

images=None

In [ ]:
classifier = load_model('model2-final.h5')

In [ ]:
# Define custom loss

def custom_loss(layer):           
     
    def loss(y_true, y_pred):       

        class_pred = classifier(y_pred)

        pred_print = ts.Print(class_pred, [class_pred], 'resnet pred: ') # PRINTS TO TERMINAL
        
        
        autoencoder_loss = losses.mean_squared_error(y_true, y_pred)

        autoencoder_mean = ts.math.reduce_mean(autoencoder_loss, [1, 2])
       
        alpha = 1
        
        final_loss = autoencoder_mean + alpha * (ts.stop_gradient(pred_print))
        
        return final_loss
  

    return loss
    


In [ ]:
# COMPILE MODEL
doggifier = Model(input_img, decoded)
batch_size= 128


doggifier.compile(optimizer='adadelta',
              loss=custom_loss(decoded), # Call the loss function with the selected layer
              metrics=['accuracy'])

In [ ]:
# TRAIN MODEL

TRAINING_LOGS_FILE = 'auto-our-classif-training.csv'

epochs=50

for j in range(epochs):
    
    print(f'Running epoch {j+1}/{epochs}')
    
    doggifier.fit(x_train, x_train,
                epochs=1,
                batch_size=128,
                shuffle=True,
                callbacks=[ CSVLogger(TRAINING_LOGS_FILE, append=False, separator=";")])
    
    epoch_number = j+1
    
    decoded_imgs = doggifier.predict(x_test)
    
    n = 4
    plt.figure(figsize=(20, 12))
    
    for i in range(n):
        
        # display original
        ax = plt.subplot(2, n, i+1)
        plt.imshow(x_test[i].reshape(64, 64,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        # display reconstruction
        ax = plt.subplot(2, n, i +1 + n)
        plt.imshow(decoded_imgs[i].reshape(64, 64,3))
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    
    fig_name = f'fig_epoch{j}-auto-our-classif.png'
    plt.savefig(fig_name)
    
    plt.show()

In [ ]:
doggifier.save('doggif_our_classif.h5')

# Test model on another set of images

In [ ]:
imgs2 = get_training_data('../data/test2/human')

In [ ]:
imgs2 = imgs2.astype('float32') / 255.
imgs2.shape

In [ ]:
decoded2 = doggifier.predict(imgs2)

In [ ]:
decoded2.shape

In [ ]:
n = 4
plt.figure(figsize=(20, 8))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(imgs2[i].reshape(64, 64,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i +1 + n)
    plt.imshow(decoded2[i].reshape(64, 64,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:


def get_training_data(datafolder):
        print("Loading training data...")

        training_data = []
        #Finds all files in datafolder
        foldernames = os.listdir(datafolder)
#         print(foldernames)
        for foldername in tqdm(foldernames):
            folder_path = os.path.join(datafolder, foldername)
            filenames = os.listdir(folder_path)
            folder = foldername
            for filename in tqdm(filenames):
                
                #Combines folder name and file name.
                path = os.path.join(datafolder,folder, filename)
                print(path)
                
                #Opens an image as an Image object.
                image = Image.open(path)
                
                #Resizes to a desired size.
                image = image.resize((64,64),Image.ANTIALIAS)
                
                #Creates an array of pixel values from the image.
                pixel_array = np.asarray(image)
#                 print(type(pixel_array))
                print(pixel_array.shape)
                training_data.append(pixel_array)

        #training_data is converted to a numpy array
        training_data = np.asarray(training_data)
#         training_data = np.reshape(training_data,(-1,64,64,1))
        return training_data